In [4]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 2.8 MB/s eta 0:00:0000:0100:01


In [1]:
import sys
configDir =  '/home/thibaud/repos/anaconda/rakuten/notebook'
if configDir not in sys.path:
    sys.path.append(configDir)

import config

print(sys.path)

['/home/thibaud/repos/anaconda/rakuten/notebook', '/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python39.zip', '/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python3.9', '/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python3.9/lib-dynload', '', '/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python3.9/site-packages', '/home/thibaud/repos/anaconda/rakuten', '/home/thibaud/repos/anaconda/rakuten/src']


In [4]:
import os
import sys
projectDir = '/home/thibaud/repos/anaconda/rakuten'
if projectDir not in sys.path:
    sys.path.append(projectDir)

import multiprocessing
num_cores = multiprocessing.cpu_count()

#Define path in the config file
import config

import pandas as pd
import numpy as np
import tensorflow as tf

from src.text.classifiers import TFbertClassifier
from src.image.classifiers import ImgClassifier
from src.multimodal.classifiers import TFmultiClassifier

from src.utils.batch import fit_save_all
from src.utils.plot import plot_training_history

import datetime

def Rakuten_img_path(img_folder, imageid, productid, suffix=''):
    """ retrurns the path to the image of a given productid and imageid"""

    df = pd.DataFrame(pd.concat([imageid, productid], axis=1))

    img_path = df.apply(lambda row:
                        os.path.join(img_folder, 'image_'
                                     + str(row['imageid'])
                                     + '_product_'
                                     + str(row['productid'])
                                     + suffix
                                     + '.jpg'),
                        axis=1)

    return img_path

data_train = pd.read_csv(os.path.join(config.path_to_data, 'df_train_index.csv'))
data_train['testset'] = False
data_test = pd.read_csv(os.path.join(config.path_to_data, 'df_test_index.csv'))
data_test['testset'] = True
data = pd.concat([data_train, data_test], axis=0)

#merging text into token column
colnames = ['designation_translated', 'description_translated'] #['designation', 'description']#
data['tokens'] = data[colnames].apply(lambda row: ' '.join(s.lower() for s in row if isinstance(s, str)), axis=1)

#path to images into img_path column
data['img_path'] = Rakuten_img_path(img_folder=config.path_to_images,
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')

#labels of encoded classes
class_labels = data.groupby('prdtypedesignation')['prdtypeindex'].first().reset_index()
class_labels.index = class_labels['prdtypeindex']
class_labels = class_labels.drop(columns='prdtypeindex').sort_index()

Img_train = data.loc[~data['testset'], 'img_path']
Img_test = data.loc[data['testset'], 'img_path']

Txt_train = data.loc[~data['testset'], 'tokens']
Txt_test = data.loc[data['testset'], 'tokens']

y_train = data.loc[~data['testset'],'prdtypeindex']
y_test = data.loc[data['testset'],'prdtypeindex']

#To be fed into any of our sklearn classifiers, X_train and X_test
#should be dataframes with columns tokens and img_path
X_train = pd.DataFrame({'tokens': Txt_train, 'img_path': Img_train})
X_test = pd.DataFrame({'tokens': Txt_test, 'img_path': Img_test})

#All data for cross-validated scores
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

#Number of classes
num_classes = len(np.unique(data['prdtypeindex']))

In [5]:
from importlib import reload
import src.multimodal.classifiers as mmclf
import src.utils.load as uload
reload(mmclf)
reload(uload)



list = ['text/SVC_skipgram','text/LinearSVC_tfidf', 'text/camembert-base-ccnet', 'image/vit_b16','text/flaubert_base_uncased', 'image/ResNet152']

clfs=[]
for model in list:
    clfs.append(uload.load_classifier(model))

clf = mmclf.MetaClassifier(
    base_estimators=clfs)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)




toto /home/thibaud/repos/anaconda/rakuten/models


/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


toto /home/thibaud/repos/anaconda/rakuten/models


/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.2.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning:

toto /home/thibaud/repos/anaconda/rakuten/models
loading from Local


...

/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python3.9/site-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


loading weights from  image/vit_b16
toto /home/thibaud/repos/anaconda/rakuten/models
loading from Local


All model checkpoint layers were used when initializing TFFlaubertModel.

All the layers of TFFlaubertModel were initialized from the model checkpoint at /home/thibaud/repos/anaconda/rakuten/models/base_models/flaubert_base_uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFFlaubertModel for predictions without further training.


loading weights from  text/flaubert_base_uncased
toto /home/thibaud/repos/anaconda/rakuten/models
loading weights from  image/ResNet152


TypeError: 'MLClassifier' object is not iterable

In [ ]:
from src.utils.plot import classification_results

classification_results(y_pred, y_test, class_labels, title='Multimodal classifier', figsize=(10,10)